<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импортируем-библиотеки" data-toc-modified-id="Импортируем-библиотеки-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импортируем библиотеки</a></span></li><li><span><a href="#Загружаем-и-изучаем-данные" data-toc-modified-id="Загружаем-и-изучаем-данные-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загружаем и изучаем данные</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression-с-взвешиванием-классов" data-toc-modified-id="Logistic-Regression-с-взвешиванием-классов-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression с взвешиванием классов</a></span><ul class="toc-item"><li><span><a href="#Преобразование-текста-с-помощью-CountVectorizer" data-toc-modified-id="Преобразование-текста-с-помощью-CountVectorizer-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Преобразование текста с помощью CountVectorizer</a></span></li><li><span><a href="#Преобразование-текста-с-помощью-TfidfVectorizer" data-toc-modified-id="Преобразование-текста-с-помощью-TfidfVectorizer-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Преобразование текста с помощью TfidfVectorizer</a></span></li><li><span><a href="#Преобразование-текста-с-помощью-HashingVectorizer" data-toc-modified-id="Преобразование-текста-с-помощью-HashingVectorizer-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Преобразование текста с помощью HashingVectorizer</a></span></li></ul></li><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LightGBM</a></span><ul class="toc-item"><li><span><a href="#Преобразование-текста-с-помощью-CountVectorizer" data-toc-modified-id="Преобразование-текста-с-помощью-CountVectorizer-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Преобразование текста с помощью CountVectorizer</a></span></li><li><span><a href="#Преобразование-текста-с-помощью-TfidfVectorizer" data-toc-modified-id="Преобразование-текста-с-помощью-TfidfVectorizer-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Преобразование текста с помощью TfidfVectorizer</a></span></li><li><span><a href="#Преобразование-текста-с-помощью-HashingVectorizer" data-toc-modified-id="Преобразование-текста-с-помощью-HashingVectorizer-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Преобразование текста с помощью HashingVectorizer</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-на-тестовой-выборке" data-toc-modified-id="Проверка-на-тестовой-выборке-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка на тестовой выборке</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Нам необходимо создать инструмент для интернет-магазина «Викишоп», который будет искать токсичные комментарии и отправлять их на модерацию. Для этого нужно обучить модель классифицировать комментарии на позитивные и негативные. Точность метрики качества F1 должна быть не меньше 0.75.

## Подготовка

### Импортируем библиотеки

In [1]:
import pandas as pd 
import numpy as np 
import re
import nltk
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_sm
import spacy

from nltk.corpus import stopwords as nltk_stopwords
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier 

     --------------------------------------- 12.8/12.8 MB 46.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
RS = 12345
KFOLD = KFold(n_splits = 5, random_state = RS, shuffle = True)

### Загружаем и изучаем данные

In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [6]:
df.duplicated().sum()

0

In [7]:
df['Unnamed: 0'].value_counts()

0         1
106294    1
106287    1
106288    1
106289    1
         ..
53159     1
53160     1
53161     1
53162     1
159450    1
Name: Unnamed: 0, Length: 159292, dtype: int64

In [8]:
df=df.drop('Unnamed: 0', axis=1)

In [9]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [10]:
display(df[df['text'].str.contains(r"\b'")].head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


#### Вывод

Исходный датасет состоит из 159292 строк и 3-х столбцов:
 - Unnamed: 0 - отражает порядковый номер комментария, для исследования не несет никакой ценности, мы его удалили.
 - text - это сам комментарий, сожержит текстовое сообщение, с которым нам предстоит работать.
 - toxic - это наш целевой признак, он отражает токсичность сообщения.

Дубликатов и пропусков в исходных данных нет. (т.к. в столбце Unnamed: 0 ни одно значение не повторяется, стоит проверить дубликаты после его удаления)  
Текст комментариев содержит слова в верхнем и нижнем регистре, слэнг, сокращения слов, символы и коды. Это нужно будет обработать.  
10% комментариев помечены токсичными.

### Предобработка данных

Напишим функцию, которая приведет все символы к нижнему регистру, заменит сокращения и слэнг:

In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"it's", "it is", text) 
    text = re.sub(r"it´s", "it is", text)    
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"that's", "that is", text)    
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"there's", "there is", text)       
    text = re.sub(r"'s", " ", text)
    text = re.sub(r"'n'", " and ", text)
    text = re.sub(r"n'", "ing", text)
    text = re.sub(r"y'all", "you all", text)
    text = text.strip(' ')
    return text

In [12]:
tqdm.pandas()
#df = df[:10]
df['text'] = df['text'].progress_apply(clean_text)

100%|██████████| 159292/159292 [00:02<00:00, 60522.07it/s]


Обработаем данные с помощью NLP библиотеки SpaСy:

In [15]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def spacy(sentence):
    doc = nlp(sentence)
    return " ".join([token.lemma_ for token in doc])

df['lemma'] = df['text'].progress_apply(spacy)

100%|██████████| 159292/159292 [12:35<00:00, 210.83it/s]


In [16]:
pd.set_option('display.max_colwidth', None)
df.head()

,text,toxic,lemma
0,"explanation\nwhy the edits made under my username hardcore metallica fan were reverted? they were not vandalisms, just closure on some gas after i voted at new york dolls fac. and please do not remove the template from the talk page since i am retired now.89.205.38.27",0,"explanation \n why the edit make under my username hardcore metallica fan be revert ? they be not vandalism , just closure on some gas after I vote at new york dolls fac . and please do not remove the template from the talk page since I be retire now.89.205.38.27"
1,"d'aww! he matches this background colour i am seemingly stuck with. thanks. (talk) 21:51, january 11, 2016 (utc)",0,"d'aww ! he match this background colour I be seemingly stuck with . thank . ( talk ) 21:51 , january 11 , 2016 ( utc )"
2,"hey man, i am really not trying to edit war. it is just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. he seems to care more about the formatting than the actual info.",0,"hey man , I be really not try to edit war . it be just that this guy be constantly remove relevant information and talk to I through edit instead of my talk page . he seem to care more about the formatting than the actual info ."
3,"""\nmore\ni ca not make any real suggestions on improvement - i wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -i think the references may need tidying so that they are all in the exact same format ie date format etc. i can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nthere appears to be a backlog on articles for review so i guess there may be a delay until a reviewer turns up. it is listed in the relevant form eg wikipedia:good_article_nominations#transport """,0,""" \n more \n I can not make any real suggestion on improvement - I wonder if the section statistic should be later on , or a subsection of "" "" type of accident "" "" -i think the reference may need tidying so that they be all in the exact same format ie date format etc . I can do that later on , if no - one else do first - if you have any preference for format style on reference or want to do it yourself please let I know . \n\n there appear to be a backlog on article for review so I guess there may be a delay until a reviewer turn up . it be list in the relevant form eg wikipedia : good_article_nominations#transport """
4,"you, sir, are my hero. any chance you remember what page that is on?",0,"you , sir , be my hero . any chance you remember what page that be on ?"


Удалим данные до обработки:

In [17]:
df = df.drop(['text'], axis=1)
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159211 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   toxic   159211 non-null  int64 
 1   lemma   159211 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Подготовим признаки разобьем на выборки:

In [18]:
features = df.drop('toxic', axis=1)
target = df['toxic']

In [19]:
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size = 0.25,
    random_state = RS
)

In [20]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(119408, 1)
(119408,)
(39803, 1)
(39803,)


Проверим распределение целевого признака:

In [21]:
print(df['toxic'].value_counts(normalize=True))
print(target_train.value_counts(normalize=True))
print(target_test.value_counts(normalize=True))

0    0.898481
1    0.101519
Name: toxic, dtype: float64
0    0.898533
1    0.101467
Name: toxic, dtype: float64
0    0.898324
1    0.101676
Name: toxic, dtype: float64


Добавим стоп слова:

In [22]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lysergide\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Вывод

В ходе предобработки данных мы:
   - привели текст к нижнему регистру и преобразовали сокращения в тексте.  
   - обработали текст с помощью NLP библиотеки SpaCy.  
   - подготовили признаки и разбили данные на выборки для обучения моделей.  
   - добавили стоп слова.  

## Обучение

Мы рассмотрим 3 способа векторизации:  
1). CountVectorizer - преобразовывает текст в векторы количества слов.  
2). TfidfVectorizer - преобразовывает текст в частотные векторы слова.  
3). HashingVectorizer - преобразовывает текст в уникальные целые числа.  
  
На примере 2-х моделей: Logistic Regression с взвешиванием классов и LightGBM.

Создадим таблицу с результатами точности моделей:

In [23]:
models = pd.DataFrame(columns =   ['model_name', 'vectorizer', 'model_var', 'f1_train'])
display(models)

,model_name,vectorizer,model_var,f1_train


Напишим функцию, которая будет добавлять результат в таблицу:

In [24]:
def models_result (model_name, vectorizer, model_var, f1_train, models):
    result = [model_name, vectorizer, model_var, f1_train]
    rows = [pd.Series(result, index = models.columns)]
    models = models.append(rows, ignore_index = True)
    return(models)

### Logistic Regression с взвешиванием классов

#### Преобразование текста с помощью CountVectorizer

In [25]:
%%time

model_name = "Logistic Regression"
vectorizer = "CountVectorizer"

model_lr_count = Pipeline(
    [
        ('count',
         CountVectorizer(
             ngram_range = (1,3),
             min_df = 2,
             max_df = 0.9,
             analyzer = 'word',
             lowercase = True,
             binary = True,
             stop_words = stopwords,
             dtype = np.float32
         )
        ),
        ('clf',
         LogisticRegression(
             random_state = RS,
             max_iter = 2500
         )
        )
    ]
)

params_lr_count = {
#    'clf__C': range(1,31,1),
    'clf__C': [2],
    'clf__class_weight': ['balanced']
}

grid_lr_count = GridSearchCV(
    estimator = model_lr_count,
    param_grid = params_lr_count,
    scoring = 'f1',
    cv = KFOLD,
    n_jobs = -1,
    refit = False
)

grid_lr_count.fit(features_train['lemma'], target_train)

model_var = grid_lr_count.best_params_
f1_train = round(grid_lr_count.best_score_, 3)

models = models_result(model_name, vectorizer, model_var, f1_train, models)

models

Wall time: 1min 1s


C:\Users\Lysergide\AppData\Local\Temp\ipykernel_7732\1690345207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(rows, ignore_index = True)


,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783


#### Преобразование текста с помощью TfidfVectorizer

In [26]:
%%time

model_name = "Logistic Regression"
vectorizer = "TfidfVectorizer"

model_lr_tfidf = Pipeline(
    [
        ('tfidf',
         TfidfVectorizer(
             ngram_range = (1,3),
             min_df = 2,
             max_df = 0.9,
             use_idf = 1,
             smooth_idf = 1,
             sublinear_tf = 1,
             stop_words = stopwords
         )
        ),
        ('clf',
         LogisticRegression(
             random_state = RS,
             max_iter = 2500
         )
        )
    ]
)

params_lr_tfidf = {
#    'clf__C': range(1,31,1),
    'clf__C': [15],
    'clf__class_weight': ['balanced']
}

grid_lr_tfidf = GridSearchCV(
    estimator = model_lr_tfidf,
    param_grid = params_lr_tfidf,
    scoring = 'f1',
    cv = KFOLD,
    n_jobs = -1,
    refit = False
)

grid_lr_tfidf.fit(features_train['lemma'], target_train)

model_var = grid_lr_tfidf.best_params_
f1_train = round(grid_lr_tfidf.best_score_, 3)

models = models_result(model_name, vectorizer, model_var, f1_train, models)

models

Wall time: 55.8 s


C:\Users\Lysergide\AppData\Local\Temp\ipykernel_7732\1690345207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(rows, ignore_index = True)


,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783
1,Logistic Regression,TfidfVectorizer,"{'clf__C': 15, 'clf__class_weight': 'balanced'}",0.783


#### Преобразование текста с помощью HashingVectorizer

In [27]:
%%time

model_name = "Logistic Regression"
vectorizer = "HashingVectorizer"

model_lr_hashing = Pipeline(
    [
        ('hashing',
         HashingVectorizer(
             ngram_range = (1,3),
             analyzer = 'word',
             lowercase = True,
             binary = True,
             stop_words = stopwords,
             dtype = np.float32
         )
        ),
        ('clf',
         LogisticRegression(
             random_state = RS,
             max_iter = 2500
         )
        )
    ]
)

params_lr_hashing = {
#    'clf__C': range(1,31,1),
    'clf__C': [13],
    'clf__class_weight': ['balanced']
}

grid_lr_hashing = GridSearchCV(
    estimator = model_lr_hashing,
    param_grid = params_lr_hashing,
    scoring = 'f1',
    cv = KFOLD,
    n_jobs = -1,
    refit = False
)

grid_lr_hashing.fit(features_train['lemma'], target_train)

model_var = grid_lr_hashing.best_params_
f1_train = round(grid_lr_hashing.best_score_, 3)

models = models_result(model_name, vectorizer, model_var, f1_train, models)

models

Wall time: 2min 46s


C:\Users\Lysergide\AppData\Local\Temp\ipykernel_7732\1690345207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(rows, ignore_index = True)


,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783
1,Logistic Regression,TfidfVectorizer,"{'clf__C': 15, 'clf__class_weight': 'balanced'}",0.783
2,Logistic Regression,HashingVectorizer,"{'clf__C': 13, 'clf__class_weight': 'balanced'}",0.780


### LightGBM

#### Преобразование текста с помощью CountVectorizer

In [28]:
%%time

model_name = "LightGBM"
vectorizer = "CountVectorizer"

model_lgbm_count = Pipeline(
    [
        ('count',
         CountVectorizer(
             ngram_range = (1,3),
             min_df = 2,
             max_df = 0.9,
             analyzer = 'word',
             lowercase = True,
             binary = True,
             stop_words = stopwords,
             dtype=np.float32
         )
        ),
        ('clf', LGBMClassifier(random_state=RS, max_iter = 2500)
        )
    ]
)

params_lgbm_count = {
  'clf__n_estimators': [100],
  'clf__learning_rate': [0.1],
  'clf__max_depth': [-1]}

grid_lgbm_count = GridSearchCV(
    estimator=model_lgbm_count,
    param_grid=params_lgbm_count,
    cv=KFOLD,
    scoring='f1',
    n_jobs=-1,
    refit=False
)
grid_lgbm_count.fit(features_train['lemma'], target_train)

model_var = grid_lgbm_count.best_params_
f1_train = round(grid_lgbm_count.best_score_, 3)

models = models_result(model_name, vectorizer, model_var, f1_train, models)

models

Wall time: 3min 35s


C:\Users\Lysergide\AppData\Local\Temp\ipykernel_7732\1690345207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(rows, ignore_index = True)


,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783
1,Logistic Regression,TfidfVectorizer,"{'clf__C': 15, 'clf__class_weight': 'balanced'}",0.783
2,Logistic Regression,HashingVectorizer,"{'clf__C': 13, 'clf__class_weight': 'balanced'}",0.780
3,LightGBM,CountVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 100}",0.776


#### Преобразование текста с помощью TfidfVectorizer

In [29]:
%%time

model_name = "LightGBM"
vectorizer = "TfidfVectorizer"

model_lgbm_tfidf = Pipeline(
    [
        ('tfidf',
         TfidfVectorizer(
             ngram_range = (1,3),
             min_df = 2,
             max_df = 0.9,
             use_idf = 1,
             smooth_idf = 1,
             sublinear_tf = 1,
             stop_words = stopwords
         )
        ),
        ('clf', LGBMClassifier(random_state=RS, max_iter = 2500)
        )
    ]
)

params_lgbm_tfidf = {
  'clf__n_estimators': [500],
  'clf__learning_rate': [0.1],
  'clf__max_depth': [-1]}

grid_lgbm_tfidf = GridSearchCV(
    estimator=model_lgbm_tfidf,
    param_grid=params_lgbm_tfidf,
    cv=KFOLD,
    scoring='f1',
    n_jobs=-1,
    refit=False
)
grid_lgbm_tfidf.fit(features_train['lemma'], target_train)

model_var = grid_lgbm_tfidf.best_params_
f1_train = round(grid_lgbm_tfidf.best_score_, 3)

models = models_result(model_name, vectorizer, model_var, f1_train, models)

models

Wall time: 14min 24s


C:\Users\Lysergide\AppData\Local\Temp\ipykernel_7732\1690345207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(rows, ignore_index = True)


,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783
1,Logistic Regression,TfidfVectorizer,"{'clf__C': 15, 'clf__class_weight': 'balanced'}",0.783
2,Logistic Regression,HashingVectorizer,"{'clf__C': 13, 'clf__class_weight': 'balanced'}",0.780
3,LightGBM,CountVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 100}",0.776
4,LightGBM,TfidfVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 500}",0.768


#### Преобразование текста с помощью HashingVectorizer

In [30]:
%%time

model_name = "LightGBM"
vectorizer = "HashingVectorizer"

model_lgbm_hashing = Pipeline(
    [
        ('hashing',
         HashingVectorizer(
             ngram_range = (1,3),
             analyzer = 'word',
             lowercase = True,
             binary = True,
             stop_words = stopwords,
             dtype = np.float32
         )
        ),
        ('clf', LGBMClassifier(random_state=RS, max_iter = 2500)
        )
    ]
)

params_lgbm_hashing = {
  'clf__n_estimators': [500],
  'clf__learning_rate': [0.1],
  'clf__max_depth': [-1]}

grid_lgbm_hashing = GridSearchCV(
    estimator=model_lgbm_hashing,
    param_grid=params_lgbm_hashing,
    cv=KFOLD,
    scoring='f1',
    n_jobs=-1,
    refit=False
)
grid_lgbm_hashing.fit(features_train['lemma'], target_train)

model_var = grid_lgbm_hashing.best_params_
f1_train = round(grid_lgbm_hashing.best_score_, 3)

models = models_result(model_name, vectorizer, model_var, f1_train, models)

models

Wall time: 16min 42s


C:\Users\Lysergide\AppData\Local\Temp\ipykernel_7732\1690345207.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models = models.append(rows, ignore_index = True)


,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783
1,Logistic Regression,TfidfVectorizer,"{'clf__C': 15, 'clf__class_weight': 'balanced'}",0.783
2,Logistic Regression,HashingVectorizer,"{'clf__C': 13, 'clf__class_weight': 'balanced'}",0.780
3,LightGBM,CountVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 100}",0.776
4,LightGBM,TfidfVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 500}",0.768
5,LightGBM,HashingVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 500}",0.768


### Вывод

Мы обучили 2 модели с использованием 3-х разных векторизаций для каждой.

In [31]:
models.sort_values('f1_train', ascending=False)

,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783
1,Logistic Regression,TfidfVectorizer,"{'clf__C': 15, 'clf__class_weight': 'balanced'}",0.783
2,Logistic Regression,HashingVectorizer,"{'clf__C': 13, 'clf__class_weight': 'balanced'}",0.780
3,LightGBM,CountVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 100}",0.776
4,LightGBM,TfidfVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 500}",0.768
5,LightGBM,HashingVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 500}",0.768


Лучшее значение на тренировочной выборке показала модель: Logistic Regression с взвешиванием классов. Исследование показало, что выбор способа векторизации практически не влияет на точность модели, поэтому тестировать придется все 3 модели.

## Проверка на тестовой выборке

Напишим функцию для тестирования моделей на тестовой выборке

In [36]:
def model_test (model_id, model_name, vectorizer):
#    test_predict = model_id.estimator.set_params(**model_id.best_params_).predict(features_test['lemma'])
    test_predict = model_id.fit(features_train['lemma'], target_train).predict(features_test['lemma'])
    f1_test = f1_score(target_test, test_predict)
    return (model_name, vectorizer, round(f1_test,3))

In [37]:
model_test(model_lr_tfidf, "Logistic Regression", "TfidfVectorizer")

('Logistic Regression', 'TfidfVectorizer', 0.707)

In [38]:
model_test(model_lr_count, "Logistic Regression", "CountVectorizer")

('Logistic Regression', 'CountVectorizer', 0.778)

In [39]:
model_test(model_lr_hashing, "Logistic Regression", "HashingVectorizer")

('Logistic Regression', 'HashingVectorizer', 0.665)

Лучшую точность метрики качества F1 (0.778) на тестовой выборке показала модель: Logistic Regression с векторизатором CountVectorizer. Это удовлетворяет условию задачи. 

## Выводы

Исходный датасет состоит из 159292 строк и 3-х столбцов:
 - Unnamed: 0 - отражает порядковый номер комментария, для исследования не несет никакой ценности, мы его удалили.
 - text - это сам комментарий, сожержит текстовое сообщение, с которым нам предстоит работать.
 - toxic - это наш целевой признак, он отражает токсичность сообщения.

Пропусков в исходных данных нет. Дубликаты были удалены.   
Текст комментариев содержит слова в верхнем и нижнем регистре, слэнг, сокращения слов, символы и коды. Это нужно будет обработать.  
10% комментариев помечены токсичными.

В ходе предобработки данных мы:
   - привели текст к нижнему регистру и преобразовали сокращения в тексте.  
   - перевели текст в формат Unicode.  
   - обработали текст с помощью NLP библиотеки SpaCy.  
   - подготовили признаки и разбили данные на выборки для обучения моделей.  
   - добавили стоп слова.  

Мы обучили 2 модели с использованием 3-х разных векторизаций для каждой.

In [40]:
models.sort_values('f1_train', ascending=False)

,model_name,vectorizer,model_var,f1_train
0,Logistic Regression,CountVectorizer,"{'clf__C': 2, 'clf__class_weight': 'balanced'}",0.783
1,Logistic Regression,TfidfVectorizer,"{'clf__C': 15, 'clf__class_weight': 'balanced'}",0.783
2,Logistic Regression,HashingVectorizer,"{'clf__C': 13, 'clf__class_weight': 'balanced'}",0.780
3,LightGBM,CountVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 100}",0.776
4,LightGBM,TfidfVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 500}",0.768
5,LightGBM,HashingVectorizer,"{'clf__learning_rate': 0.1, 'clf__max_depth': -1, 'clf__n_estimators': 500}",0.768


Лучшее значение на тренировочной выборке показала модель: Logistic Regression с взвешиванием классов. Исследование показало, что выбор способа векторизации практически не влияет на точность модели, поэтому тестировать придется все 3 модели.

Лучшую точность метрики качества F1 (0.778) на тестовой выборке показала модель: Logistic Regression с векторизатором CountVectorizer. Это удовлетворяет условию задачи. 